# Import necessary packages

In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from random import sample

import re

import xgboost as xgb

from sklearn.model_selection import RandomizedSearchCV

from timeit import default_timer as timer

from sklearn.metrics import mean_absolute_error, mean_squared_error

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import original data and preprocessed data

In [3]:
sales_train_val = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/sales_train_validation.csv')
calendar = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/calendar.csv')

In [4]:
sales_train_val = sales_train_val[(sales_train_val['state_id'] == 'CA') & (sales_train_val['cat_id'] == 'HOUSEHOLD')]

In [5]:
INPUT_DIR_2 = '/content/drive/MyDrive/Colab Notebooks/List of Product ID according to 4 demand patterns/California/Household/2-year'
list_intermittent = pd.read_csv(f'{INPUT_DIR_2}/Intermittent_ID_2_Year_Data.csv')
list_lumpy = pd.read_csv(f'{INPUT_DIR_2}/Lumpy_ID_2_Year_Data.csv')
list_erratic = pd.read_csv(f'{INPUT_DIR_2}/Erratic_ID_2_Year_Data.csv')
list_smooth = pd.read_csv(f'{INPUT_DIR_2}/Smooth_ID_2_Year_Data.csv')

list_intermittent = list_intermittent['0'].values.tolist()
list_lumpy = list_lumpy['0'].values.tolist()
list_erratic = list_erratic['0'].values.tolist()
list_smooth = list_smooth['0'].values.tolist()

sales_intermittent = sales_train_val[sales_train_val.id.isin(list_intermittent)]
sales_lumpy = sales_train_val[sales_train_val.id.isin(list_lumpy)]
sales_erratic = sales_train_val[sales_train_val.id.isin(list_erratic)]
sales_smooth = sales_train_val[sales_train_val.id.isin(list_smooth)]

In [6]:
sales_smooth

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
571,HOUSEHOLD_1_007_CA_1_validation,HOUSEHOLD_1_007,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,0,3,1,1,1,3,0,5,3,2
583,HOUSEHOLD_1_019_CA_1_validation,HOUSEHOLD_1_019,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,2,5,1,1,1,0,0,0,2,0
591,HOUSEHOLD_1_027_CA_1_validation,HOUSEHOLD_1_027,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,4,0,0,0,...,1,3,4,2,1,1,2,3,1,3
601,HOUSEHOLD_1_038_CA_1_validation,HOUSEHOLD_1_038,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,3,3,2,2,...,0,1,1,2,4,1,3,1,2,1
602,HOUSEHOLD_1_039_CA_1_validation,HOUSEHOLD_1_039,HOUSEHOLD_1,HOUSEHOLD,CA_1,CA,0,0,0,0,...,1,3,1,1,0,1,2,1,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10216,HOUSEHOLD_1_514_CA_4_validation,HOUSEHOLD_1_514,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,0,0,0,0,...,1,0,5,2,3,0,0,0,2,2
10219,HOUSEHOLD_1_517_CA_4_validation,HOUSEHOLD_1_517,HOUSEHOLD_1,HOUSEHOLD,CA_4,CA,0,0,0,0,...,0,3,6,3,9,2,2,7,1,10
10287,HOUSEHOLD_2_044_CA_4_validation,HOUSEHOLD_2_044,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,1,2,6,2,1,2,3,4,2,5
10353,HOUSEHOLD_2_110_CA_4_validation,HOUSEHOLD_2_110,HOUSEHOLD_2,HOUSEHOLD,CA_4,CA,0,0,0,0,...,2,2,3,4,2,2,2,3,3,2


# User-defined functions to calculate Metrics

In [7]:
ROUNDING_DECIMAL = 4

def mase_calculation(ts, prediction):
    divisor = 0
    for i in range(1, ts.shape[0]):
        divisor = divisor + abs(ts.iloc[i] - ts.iloc[i-1])
    divisor = divisor/(ts.shape[0] - 1)
    diff    = abs(ts - prediction[:ts.shape[0]])/divisor
    mase    = diff.mean()
    return mase

def mape_calculation(actual, pred): 
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    mask = (actual != 0)
    return round((np.fabs(actual - pred)/actual)[mask].mean()*100, ROUNDING_DECIMAL)

def wmape_calculation(actual, pred):
    if not all([isinstance(actual, np.ndarray), isinstance(pred, np.ndarray)]):
        actual, pred = np.array(actual), np.array(pred)
    return round((np.sum(np.absolute(actual-pred))/np.sum(actual))*100, ROUNDING_DECIMAL)

def smape_calculation(actual, predicted):
    if not all([isinstance(actual, np.ndarray), isinstance(predicted, np.ndarray)]):
        actual, predicted = np.array(actual), np.array(predicted)
    return round(np.mean(np.abs(predicted - actual) / ((np.abs(predicted) + np.abs(actual))/2))*100, ROUNDING_DECIMAL)

# Format sales data and then merge with calendar data

In [8]:
def data_cleaning(sales_pattern):

    sales_pattern_py = sales_pattern.copy()
    list_pattern_py = sales_pattern_py.id.unique().tolist()

    sales_pattern_py = sales_pattern_py.drop(['item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'], axis=1)
    df_pattern = sales_pattern_py.melt(['id'], var_name='Date').pivot(index = ['Date'], columns = 'id', values = 'value').reset_index()
    df_pattern.drop('Date', axis=1, inplace=True)
    df_pattern.index = pd.date_range('2011-01-29', periods=df_pattern.shape[0], freq="D")
    df_pattern.index.names = ['Date']
    df_pattern = df_pattern.astype('float64')

    df_pattern.iloc[:, 1:].columns = df_pattern.iloc[:, 1:].columns.str.rstrip('_validation')
    df_pattern.columns = df_pattern.columns.str.replace(r'_validation', '')
    df_pattern.reset_index(inplace=True)

    calendar['Date'] = pd.to_datetime(calendar['date'])

    dataframe = pd.merge(df_pattern,
                         calendar[['wday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'Date']],
                         on = 'Date')

    dataframe['weekend'] = np.where(((dataframe['wday'] == 1) | (dataframe['wday'] == 2)), 1, 0)
    dataframe.fillna('No event', inplace=True)
    dataframe['year'] = dataframe['year'].apply(str)

    dataframe['quarter'] = dataframe['Date'].dt.quarter
    dataframe['quarter'] = dataframe['quarter'].apply(str)

    dataframe = pd.get_dummies(dataframe)
    
    return dataframe

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---


In [9]:
data_pattern = data_cleaning(sales_smooth)

In [10]:
data_pattern

,Date,HOUSEHOLD_1_003_CA_3,HOUSEHOLD_1_004_CA_3,HOUSEHOLD_1_007_CA_1,HOUSEHOLD_1_007_CA_3,HOUSEHOLD_1_018_CA_3,HOUSEHOLD_1_019_CA_1,HOUSEHOLD_1_019_CA_2,HOUSEHOLD_1_019_CA_3,HOUSEHOLD_1_027_CA_1,...,event_name_1_VeteransDay,event_type_1_Cultural,event_type_1_National,event_type_1_No event,event_type_1_Religious,event_type_1_Sporting,quarter_1,quarter_2,quarter_3,quarter_4
0,2011-01-29,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0,0,0,1,0,0,1,0,0,0
1,2011-01-30,0.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0,0,0,1,0,0,1,0,0,0
2,2011-01-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0,0,0,1,0,0,1,0,0,0
3,2011-02-01,0.0,3.0,4.0,3.0,0.0,5.0,6.0,27.0,1.0,...,0,0,0,1,0,0,1,0,0,0
4,2011-02-02,0.0,4.0,3.0,1.0,0.0,2.0,11.0,30.0,2.0,...,0,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1908,2016-04-20,2.0,1.0,1.0,1.0,0.0,5.0,15.0,34.0,5.0,...,0,0,0,1,0,0,0,1,0,0
1909,2016-04-21,0.0,11.0,2.0,0.0,0.0,8.0,9.0,17.0,6.0,...,0,0,0,1,0,0,0,1,0,0
1910,2016-04-22,0.0,3.0,3.0,4.0,0.0,5.0,2.0,37.0,6.0,...,0,0,0,1,0,0,0,1,0,0
1911,2016-04-23,0.0,5.0,1.0,2.0,0.0,5.0,4.0,36.0,1.0,...,0,0,0,1,0,0,0,1,0,0


# XGBoost Hyperparameters tuning to find optimum parameters

In [21]:
not_id_cols = [
 'Date',
 'wday',
 'month',
 'snap_CA',
 'weekend',
 'year_2011',
 'year_2012',
 'year_2013',
 'year_2014',
 'year_2015',
 'year_2016',
 'event_name_1_Chanukah End',
 'event_name_1_Christmas',
 'event_name_1_Cinco De Mayo',
 'event_name_1_ColumbusDay',
 'event_name_1_Easter',
 'event_name_1_Eid al-Fitr',
 'event_name_1_EidAlAdha',
 "event_name_1_Father's day",
 'event_name_1_Halloween',
 'event_name_1_IndependenceDay',
 'event_name_1_LaborDay',
 'event_name_1_LentStart',
 'event_name_1_LentWeek2',
 'event_name_1_MartinLutherKingDay',
 'event_name_1_MemorialDay',
 "event_name_1_Mother's day",
 'event_name_1_NBAFinalsEnd',
 'event_name_1_NBAFinalsStart',
 'event_name_1_NewYear',
 'event_name_1_No event',
 'event_name_1_OrthodoxChristmas',
 'event_name_1_OrthodoxEaster',
 'event_name_1_Pesach End',
 'event_name_1_PresidentsDay',
 'event_name_1_Purim End',
 'event_name_1_Ramadan starts',
 'event_name_1_StPatricksDay',
 'event_name_1_SuperBowl',
 'event_name_1_Thanksgiving',
 'event_name_1_ValentinesDay',
 'event_name_1_VeteransDay',
 'event_type_1_Cultural',
 'event_type_1_National',
 'event_type_1_No event',
 'event_type_1_Religious',
 'event_type_1_Sporting',
 'quarter_1',
 'quarter_2',
 'quarter_3',
 'quarter_4'
]

In [22]:
end_train_date = '2016-04-10'

In [23]:
lags = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28]
moving_averages = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
stds = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
maximums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
minimums = [2, 3, 4, 5, 6, 7, 14, 21, 28]  
totals = [2, 3, 4, 5, 6, 7, 14, 21, 28]
num_days_zeros = [2, 3, 4, 5, 6, 7, 14, 21, 28] 

In [24]:
correlation_threshold = 0.8

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



# Decide optimum set of parameters for XGBoost algorithm

# Implement XGBoost on Test Set with optimum set of parameters

In [25]:
subsample_chosen = 0.7
n_estimators_chosen = 50
max_depth_chosen = 3
learning_rate_chosen = 0.1               
colsample_bytree_chosen =  0.8
colsample_bylevel_chosen = 0.6

In [26]:
# User-defined function to run XGBoost on Test Data

def XGBoost_Test_Data(list_pattern, not_id_cols, lags, moving_averages, stds,
                      maximums, minimums, totals, num_days_zeros, correlation_threshold,
                      end_train_date, subsample_chosen, n_estimators_chosen,
                      max_depth_chosen, learning_rate_chosen, 
                      colsample_bytree_chosen, colsample_bylevel_chosen):
    
    list_pattern_temp = list_pattern.copy()
    list_pattern_temp = [(re.sub(r'_validation', r"", x)) for x in list_pattern_temp]
    
    df_output_xgb_pattern = pd.DataFrame()
    
    for product in list_pattern_temp:
        print('Currently Running Product: %s' % product)
        print('Progressing: {0} %'.format(round(list_pattern_temp.index(product) / len(list_pattern_temp) * 100, 2)))

        dataframe_product = pd.concat([data_pattern[[product]], data_pattern[not_id_cols]], axis=1, ignore_index=False)

        # Create Lag variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (t-{lag})': dataframe_product[product].shift(lag)
            for lag in lags
        })

        # Create Rolling Moving Average variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (mva_{moving_average})': dataframe_product[product].shift(1).rolling(moving_average).mean()
            for moving_average in moving_averages
        })

        # Create Rolling Standard Deviation variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (std_{std})': dataframe_product[product].shift(1).rolling(std).std()
            for std in stds
        })

        # Create Rolling Maximum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (max_{maximum})': dataframe_product[product].shift(1).rolling(maximum).max()
            for maximum in maximums
        })

        # Create Rolling Minimum variables
        dataframe_product = dataframe_product.assign(**{
            f'{product} (min_{minimum})': dataframe_product[product].shift(1).rolling(minimum).min()
            for minimum in minimums
        })

        # Create Rolling Total variables 
        dataframe_product = dataframe_product.assign(**{
            f'{product} (total_{total})': dataframe_product[product].shift(1).rolling(total).sum()
            for total in totals
        })

        # Create Zero Count variables
        count_func = lambda x: (x==0).sum()

        dataframe_product = dataframe_product.assign(**{
            f'{product} (zero_count_in_{num_days_zero})': dataframe_product[product].shift(1).rolling(num_days_zero).apply(count_func)
            for num_days_zero in num_days_zeros
        })
        
        dataframe_product.dropna(inplace=True)
        dataframe_product.set_index('Date', inplace=True)

        train_data = dataframe_product[dataframe_product.index <= end_train_date]
        test_data = dataframe_product[dataframe_product.index > end_train_date]

        corr_matrix = train_data.loc[:, train_data.columns != product].corr().abs()
        high_corr_var=np.where(corr_matrix > correlation_threshold)
        high_corr_var=[(corr_matrix.columns[x],corr_matrix.columns[y]) for x,y in zip(*high_corr_var) if x!=y and x<y]
        
        train_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        test_data.drop([i[1] for i in high_corr_var], axis=1, inplace=True)
        
        X_train = train_data.drop(product, axis=1)
        y_train = train_data[product]

        X_test = test_data.drop(product, axis=1)
        y_test = test_data[product]

        xgb_r = xgb.XGBRegressor(seed = 20,
                                subsample = subsample_chosen,
                                n_estimators = n_estimators_chosen,
                                max_depth = max_depth_chosen,
                                learning_rate = learning_rate_chosen,               
                                colsample_bytree = colsample_bytree_chosen,
                                colsample_bylevel = colsample_bylevel_chosen)
        

        xgb_r.fit(X_train,y_train)
        y_pred = xgb_r.predict(X_test)
        df_output_temp = pd.DataFrame({'Actual Data': y_test, 
                                       'Forecast': y_pred, 
                                       'Product': [product for num_rows in range(len(y_pred))]
                                       })
        
        df_output_xgb_pattern = df_output_xgb_pattern.append(df_output_temp, ignore_index=False)
        
    return df_output_xgb_pattern

***Decide which pattern we are dealing with?:***

---
INTERMITTENT
---
LUMPY
---
ERRATIC
---
SMOOTH
---



In [27]:
# Start running XGBoost on Test Set

start = timer()

df_forecast_values = XGBoost_Test_Data(list_smooth, not_id_cols, lags, moving_averages, stds,
                                      maximums, minimums, totals, num_days_zeros, correlation_threshold,
                                      end_train_date, subsample_chosen, n_estimators_chosen,
                                      max_depth_chosen, learning_rate_chosen, 
                                      colsample_bytree_chosen, colsample_bylevel_chosen)

end = timer()

print('This line of code took {} minutes'.format((end-start) / 60))

Currently Running Product: HOUSEHOLD_1_003_CA_3
Progressing: 0.0 %
[03:31:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running Product: HOUSEHOLD_1_004_CA_3
Progressing: 0.26 %
[03:31:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running Product: HOUSEHOLD_1_007_CA_1
Progressing: 0.52 %
[03:32:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running Product: HOUSEHOLD_1_007_CA_3
Progressing: 0.78 %
[03:32:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running Product: HOUSEHOLD_1_018_CA_3
Progressing: 1.04 %
[03:32:08] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
Currently Running Product:

In [28]:
df_forecast_values

,Actual Data,Forecast,Product
Date,,,
2016-04-11,0.0,1.759246,HOUSEHOLD_1_003_CA_3
2016-04-12,2.0,1.494351,HOUSEHOLD_1_003_CA_3
2016-04-13,3.0,1.198929,HOUSEHOLD_1_003_CA_3
2016-04-14,0.0,1.468169,HOUSEHOLD_1_003_CA_3
2016-04-15,1.0,1.659720,HOUSEHOLD_1_003_CA_3
...,...,...,...
2016-04-20,20.0,0.347893,HOUSEHOLD_2_513_CA_3
2016-04-21,8.0,2.676142,HOUSEHOLD_2_513_CA_3
2016-04-22,2.0,2.826040,HOUSEHOLD_2_513_CA_3


In [29]:
df_forecast_values.isnull().sum()

Actual Data    0
Forecast       0
Product        0
dtype: int64

In [30]:
df_forecast_values.to_csv('XGBoost_Smooth_Forecast_Values.csv')

In [31]:
# User-defined funcion to calculate metrics

def get_metrics_result_all_params(data):
    metrics_df = {}
    metrics_df['MASE'] = mase_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['WMAPE'] = wmape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['SMAPE'] = smape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAPE'] = mape_calculation(data['Actual Data'], data['Forecast'])
    metrics_df['MAE'] = mean_absolute_error(data['Actual Data'], data['Forecast'])
    metrics_df['RMSE'] = np.sqrt(mean_squared_error(data['Actual Data'], data['Forecast']))
    return pd.Series(metrics_df)

In [32]:
# Start calculating metrics 
start = timer()

df_forecast_metrics = df_forecast_values.groupby('Product').apply(get_metrics_result_all_params).reset_index()

end = timer()
print('This line of code took {} minutes'.format((end-start) / 60))

This line of code took 0.013286706550002237 minutes


In [33]:
df_forecast_metrics

,Product,MASE,WMAPE,SMAPE,MAPE,MAE,RMSE
0,HOUSEHOLD_1_003_CA_3,0.969366,167.0292,150.8847,50.8537,1.193066,1.295732
1,HOUSEHOLD_1_004_CA_3,0.808610,63.1765,75.5175,62.2577,2.301429,3.048041
2,HOUSEHOLD_1_007_CA_1,1.382985,67.0216,60.1486,75.7040,0.957451,1.106557
3,HOUSEHOLD_1_007_CA_3,0.799649,96.5543,104.2714,75.2588,2.275924,2.576973
4,HOUSEHOLD_1_018_CA_3,inf,inf,200.0000,NaN,0.036829,0.036836
...,...,...,...,...,...,...,...
381,HOUSEHOLD_2_483_CA_3,0.557916,50.0694,54.6126,43.8027,1.716665,2.587169
382,HOUSEHOLD_2_490_CA_2,inf,inf,200.0000,NaN,0.013425,0.013425
383,HOUSEHOLD_2_490_CA_3,inf,inf,200.0000,NaN,0.014785,0.014785
384,HOUSEHOLD_2_510_CA_3,0.688432,43.6110,49.9666,37.2944,1.059125,1.392400


In [34]:
df_forecast_values.to_csv('XGBoost_Smooth_Forecast_Metrics.csv')